In [1]:
import findspark
findspark.init()

In [16]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd

In [3]:
spark = SparkSession.builder.appName('Kishor').getOrCreate()
spark

In [5]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]
m_schema = ['id','name','age','salary', 'country','dept']
df = spark.createDataFrame(data = emp_data, schema=m_schema)
df.show()

+----+-------+----+------+-------+-----------+
|  id|   name| age|salary|country|       dept|
+----+-------+----+------+-------+-----------+
|   1| manish|  26| 20000|  india|         IT|
|   2|  rahul|NULL| 40000|germany|engineering|
|   3|  pawan|  12| 60000|  india|      sales|
|   4|roshini|  44|  NULL|     uk|engineering|
|   5|raushan|  35| 70000|  india|      sales|
|   6|   NULL|  29|200000|     uk|         IT|
|   7|   adam|  37| 65000|     us|         IT|
|   8|  chris|  16| 40000|     us|      sales|
|NULL|   NULL|NULL|  NULL|   NULL|       NULL|
|   7|   adam|  37| 65000|     us|         IT|
+----+-------+----+------+-------+-----------+



In [6]:
df.count()

10

In [9]:
df.select(count('name')).show()
df.select(count('id')).show()

+-----------+
|count(name)|
+-----------+
|          8|
+-----------+

+---------+
|count(id)|
+---------+
|        9|
+---------+



In [10]:
df.select(count('*')).show()

+--------+
|count(1)|
+--------+
|      10|
+--------+



In [12]:
df.select(sum('salary').alias("sum_of_salary"), max('salary'), min('salary')).show()

+-------------+-----------+-----------+
|sum_of_salary|max(salary)|min(salary)|
+-------------+-----------+-----------+
|       560000|     200000|      20000|
+-------------+-----------+-----------+



In [14]:
df.select(sum('salary').alias("sum_of_salary"), avg('salary').cast('int').alias('avg_salary')).show()

+-------------+----------+
|sum_of_salary|avg_salary|
+-------------+----------+
|       560000|     70000|
+-------------+----------+



In [16]:
data1 = [(1,'manish',50000,'IT'),
(2,'vikash',60000,'sales'),
(3,'raushan',70000,'marketing'),
(4,'mukesh',80000,'IT'),
(5,'pritam',90000,'sales'),
(6,'nikita',45000,'marketing'),
(7,'ragini',55000,'marketing'),
(8,'rakesh',100000,'IT'),
(9,'aditya',65000,'IT'),
(10,'rahul',50000,'marketing')]

m_schema = ['id', 'name', 'salary', 'dept']
df = spark.createDataFrame(data = data1, schema=m_schema)
df.show()

+---+-------+------+---------+
| id|   name|salary|     dept|
+---+-------+------+---------+
|  1| manish| 50000|       IT|
|  2| vikash| 60000|    sales|
|  3|raushan| 70000|marketing|
|  4| mukesh| 80000|       IT|
|  5| pritam| 90000|    sales|
|  6| nikita| 45000|marketing|
|  7| ragini| 55000|marketing|
|  8| rakesh|100000|       IT|
|  9| aditya| 65000|       IT|
| 10|  rahul| 50000|marketing|
+---+-------+------+---------+



In [17]:
df.groupBy('dept').agg(sum('salary')).show()

+---------+-----------+
|     dept|sum(salary)|
+---------+-----------+
|       IT|     295000|
|    sales|     150000|
|marketing|     220000|
+---------+-----------+



In [18]:
data2 = [(1,'manish',50000,'IT','india'),
(2,'vikash',60000,'sales','us'),
(3,'raushan',70000,'marketing','india'),
(4,'mukesh',80000,'IT','us'),
(5,'pritam',90000,'sales','india'),
(6,'nikita',45000,'marketing','us'),
(7,'ragini',55000,'marketing','india'),
(8,'rakesh',100000,'IT','us'),
(9,'aditya',65000,'IT','india'),
(10,'rahul',50000,'marketing','us') ]

m_schema1 = ['id', 'name', 'salary', 'dept', 'country']
df1 = spark.createDataFrame(data = data2, schema=m_schema1)
df1.show()

+---+-------+------+---------+-------+
| id|   name|salary|     dept|country|
+---+-------+------+---------+-------+
|  1| manish| 50000|       IT|  india|
|  2| vikash| 60000|    sales|     us|
|  3|raushan| 70000|marketing|  india|
|  4| mukesh| 80000|       IT|     us|
|  5| pritam| 90000|    sales|  india|
|  6| nikita| 45000|marketing|     us|
|  7| ragini| 55000|marketing|  india|
|  8| rakesh|100000|       IT|     us|
|  9| aditya| 65000|       IT|  india|
| 10|  rahul| 50000|marketing|     us|
+---+-------+------+---------+-------+



In [24]:
df1.groupBy('dept', 'country').agg(sum('salary')).show()

+---------+-------+-----------+
|     dept|country|sum(salary)|
+---------+-------+-----------+
|       IT|  india|     115000|
|    sales|     us|      60000|
|marketing|  india|     125000|
|    sales|  india|      90000|
|       IT|     us|     180000|
|marketing|     us|      95000|
+---------+-------+-----------+



# Join

In [4]:
customer_data = [(1,'manish','patna',"30-05-2022"),
(2,'vikash','kolkata',"12-03-2023"),
(3,'nikita','delhi',"25-06-2023"),
(4,'rahul','ranchi',"24-03-2023"),
(5,'mahesh','jaipur',"22-03-2023"),
(6,'prantosh','kolkata',"18-10-2022"),
(7,'raman','patna',"30-12-2022"),
(8,'prakash','ranchi',"24-02-2023"),
(9,'ragini','kolkata',"03-03-2023"),
(10,'raushan','jaipur',"05-02-2023")]


customer_schema=['customer_id','customer_name','address','date_of_joining']
customer_df = spark.createDataFrame(data= customer_data, schema = customer_schema)

sales_data = [(1,22,10,"01-06-2022"),
(1,27,5,"03-02-2023"),
(2,5,3,"01-06-2023"),
(5,22,1,"22-03-2023"),
(7,22,4,"03-02-2023"),
(9,5,6,"03-03-2023"),
(2,1,12,"15-06-2023"),
(1,56,2,"25-06-2023"),
(5,12,5,"15-04-2023"),
(11,12,76,"12-03-2023")]

sales_schema=['customer_id','product_id','quantity','date_of_purchase']
sales_df = spark.createDataFrame(data= sales_data, schema = sales_schema)

product_data = [(1, 'fanta',20),
(2, 'dew',22),
(5, 'sprite',40),
(7, 'redbull',100),
(12,'mazza',45),
(22,'coke',27),
(25,'limca',21),
(27,'pepsi',14),
(56,'sting',10)]

product_schema=['id','name','price']

In [5]:
customer_df.show()
sales_df.show()

+-----------+-------------+-------+---------------+
|customer_id|customer_name|address|date_of_joining|
+-----------+-------------+-------+---------------+
|          1|       manish|  patna|     30-05-2022|
|          2|       vikash|kolkata|     12-03-2023|
|          3|       nikita|  delhi|     25-06-2023|
|          4|        rahul| ranchi|     24-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|
|          6|     prantosh|kolkata|     18-10-2022|
|          7|        raman|  patna|     30-12-2022|
|          8|      prakash| ranchi|     24-02-2023|
|          9|       ragini|kolkata|     03-03-2023|
|         10|      raushan| jaipur|     05-02-2023|
+-----------+-------------+-------+---------------+

+-----------+----------+--------+----------------+
|customer_id|product_id|quantity|date_of_purchase|
+-----------+----------+--------+----------------+
|          1|        22|      10|      01-06-2022|
|          1|        27|       5|      03-02-2023|
|          2|   

In [7]:
customer_df.join(sales_df, sales_df['customer_id'] == customer_df['customer_id'], 'inner').show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15-06-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        12|       5|      15

In [9]:
customer_df.join(sales_df, sales_df['customer_id'] == customer_df['customer_id'], 'inner')\
.select(sales_df['customer_id']).show()

+-----------+
|customer_id|
+-----------+
|          1|
|          1|
|          1|
|          2|
|          2|
|          5|
|          5|
|          7|
|          9|
+-----------+



# Window

In [6]:
from pyspark.sql.window import Window

In [5]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

m_schema = ['id','name','salary','dept','gender']
df= spark.createDataFrame(data=emp_data, schema = m_schema)
df.show()

+---+--------+------+---------+------+
| id|    name|salary|     dept|gender|
+---+--------+------+---------+------+
|  1|  manish| 50000|       IT|     m|
|  2|  vikash| 60000|    sales|     m|
|  3| raushan| 70000|marketing|     m|
|  4|  mukesh| 80000|       IT|     m|
|  5|   priti| 90000|    sales|     f|
|  6|  nikita| 45000|marketing|     f|
|  7|  ragini| 55000|marketing|     f|
|  8|   rashi|100000|       IT|     f|
|  9|  aditya| 65000|       IT|     m|
| 10|   rahul| 50000|marketing|     m|
| 11|   rakhi| 50000|       IT|     f|
| 12|akhilesh| 90000|    sales|     m|
+---+--------+------+---------+------+



In [20]:
window = Window.partitionBy('dept').orderBy('salary')

df.withColumn('row_number', row_number().over(window))\
.withColumn('row', rank().over(window))\
.withColumn('dense_rank', dense_rank().over(window)).show(truncate=False)

+---+--------+------+---------+------+----------+---+----------+
|id |name    |salary|dept     |gender|row_number|row|dense_rank|
+---+--------+------+---------+------+----------+---+----------+
|1  |manish  |50000 |IT       |m     |1         |1  |1         |
|11 |rakhi   |50000 |IT       |f     |2         |1  |1         |
|9  |aditya  |65000 |IT       |m     |3         |3  |2         |
|4  |mukesh  |80000 |IT       |m     |4         |4  |3         |
|8  |rashi   |100000|IT       |f     |5         |5  |4         |
|6  |nikita  |45000 |marketing|f     |1         |1  |1         |
|10 |rahul   |50000 |marketing|m     |2         |2  |2         |
|7  |ragini  |55000 |marketing|f     |3         |3  |3         |
|3  |raushan |70000 |marketing|m     |4         |4  |4         |
|2  |vikash  |60000 |sales    |m     |1         |1  |1         |
|5  |priti   |90000 |sales    |f     |2         |2  |2         |
|12 |akhilesh|90000 |sales    |m     |3         |2  |2         |
+---+--------+------+----

In [22]:
window = Window.partitionBy('dept','gender').orderBy('salary')

df.withColumn('row_number', row_number().over(window))\
.withColumn('row', rank().over(window))\
.withColumn('dense_rank', dense_rank().over(window)).show(truncate=False) 

+---+--------+------+---------+------+----------+---+----------+
|id |name    |salary|dept     |gender|row_number|row|dense_rank|
+---+--------+------+---------+------+----------+---+----------+
|11 |rakhi   |50000 |IT       |f     |1         |1  |1         |
|8  |rashi   |100000|IT       |f     |2         |2  |2         |
|1  |manish  |50000 |IT       |m     |1         |1  |1         |
|9  |aditya  |65000 |IT       |m     |2         |2  |2         |
|4  |mukesh  |80000 |IT       |m     |3         |3  |3         |
|6  |nikita  |45000 |marketing|f     |1         |1  |1         |
|7  |ragini  |55000 |marketing|f     |2         |2  |2         |
|10 |rahul   |50000 |marketing|m     |1         |1  |1         |
|3  |raushan |70000 |marketing|m     |2         |2  |2         |
|5  |priti   |90000 |sales    |f     |1         |1  |1         |
|2  |vikash  |60000 |sales    |m     |1         |1  |1         |
|12 |akhilesh|90000 |sales    |m     |2         |2  |2         |
+---+--------+------+----

In [24]:
window = Window.partitionBy('dept').orderBy(desc('salary'))
df.withColumn('row_number', row_number().over(window))\
.withColumn('row', rank().over(window))\
.withColumn('dense_rank', dense_rank().over(window))\
.filter(col('dense_rank') <= 2).show(truncate=False)

+---+--------+------+---------+------+----------+---+----------+
|id |name    |salary|dept     |gender|row_number|row|dense_rank|
+---+--------+------+---------+------+----------+---+----------+
|8  |rashi   |100000|IT       |f     |1         |1  |1         |
|4  |mukesh  |80000 |IT       |m     |2         |2  |2         |
|3  |raushan |70000 |marketing|m     |1         |1  |1         |
|7  |ragini  |55000 |marketing|f     |2         |2  |2         |
|5  |priti   |90000 |sales    |f     |1         |1  |1         |
|12 |akhilesh|90000 |sales    |m     |2         |1  |1         |
|2  |vikash  |60000 |sales    |m     |3         |3  |2         |
+---+--------+------+---------+------+----------+---+----------+



# lead and lag in spark | window function in pyspark

In [27]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]
col_n = ['product_id','product_name', 'sales_date', 'sales']
df = spark.createDataFrame(data=product_data, schema = col_n)
df.show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         1|      iphone|01-01-2023|1500000|
|         2|     samsung|01-01-2023|1100000|
|         3|     oneplus|01-01-2023|1100000|
|         1|      iphone|01-02-2023|1300000|
|         2|     samsung|01-02-2023|1120000|
|         3|     oneplus|01-02-2023|1120000|
|         1|      iphone|01-03-2023|1600000|
|         2|     samsung|01-03-2023|1080000|
|         3|     oneplus|01-03-2023|1160000|
|         1|      iphone|01-04-2023|1700000|
|         2|     samsung|01-04-2023|1800000|
|         3|     oneplus|01-04-2023|1170000|
|         1|      iphone|01-05-2023|1200000|
|         2|     samsung|01-05-2023| 980000|
|         3|     oneplus|01-05-2023|1175000|
|         1|      iphone|01-06-2023|1100000|
|         2|     samsung|01-06-2023|1100000|
|         3|     oneplus|01-06-2023|1200000|
+----------+------------+----------+-------+



In [30]:
window = Window.partitionBy('product_id').orderBy('sales_date')
last_month_df =df.withColumn('previous_month_sales', lag(col('sales'),1).over(window))
last_month_df.show()

+----------+------------+----------+-------+--------------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|
+----------+------------+----------+-------+--------------------+
|         1|      iphone|01-01-2023|1500000|                NULL|
|         1|      iphone|01-02-2023|1300000|             1500000|
|         1|      iphone|01-03-2023|1600000|             1300000|
|         1|      iphone|01-04-2023|1700000|             1600000|
|         1|      iphone|01-05-2023|1200000|             1700000|
|         1|      iphone|01-06-2023|1100000|             1200000|
|         2|     samsung|01-01-2023|1100000|                NULL|
|         2|     samsung|01-02-2023|1120000|             1100000|
|         2|     samsung|01-03-2023|1080000|             1120000|
|         2|     samsung|01-04-2023|1800000|             1080000|
|         2|     samsung|01-05-2023| 980000|             1800000|
|         2|     samsung|01-06-2023|1100000|              980000|
|         

In [31]:
# calculate loss 
 
last_month_df.withColumn('per_loss_gain', col('sales')-col('previous_month_sales')).show()

+----------+------------+----------+-------+--------------------+-------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|per_loss_gain|
+----------+------------+----------+-------+--------------------+-------------+
|         1|      iphone|01-01-2023|1500000|                NULL|         NULL|
|         1|      iphone|01-02-2023|1300000|             1500000|      -200000|
|         1|      iphone|01-03-2023|1600000|             1300000|       300000|
|         1|      iphone|01-04-2023|1700000|             1600000|       100000|
|         1|      iphone|01-05-2023|1200000|             1700000|      -500000|
|         1|      iphone|01-06-2023|1100000|             1200000|      -100000|
|         2|     samsung|01-01-2023|1100000|                NULL|         NULL|
|         2|     samsung|01-02-2023|1120000|             1100000|        20000|
|         2|     samsung|01-03-2023|1080000|             1120000|       -40000|
|         2|     samsung|01-04-2023|1800

In [37]:
# calculate percent & round off

last_month_df.withColumn('per_loss_gain', round(((col('sales')-col('previous_month_sales'))/col('sales'))*100,2)).show()

+----------+------------+----------+-------+--------------------+-------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|per_loss_gain|
+----------+------------+----------+-------+--------------------+-------------+
|         1|      iphone|01-01-2023|1500000|                NULL|         NULL|
|         1|      iphone|01-02-2023|1300000|             1500000|       -15.38|
|         1|      iphone|01-03-2023|1600000|             1300000|        18.75|
|         1|      iphone|01-04-2023|1700000|             1600000|         5.88|
|         1|      iphone|01-05-2023|1200000|             1700000|       -41.67|
|         1|      iphone|01-06-2023|1100000|             1200000|        -9.09|
|         2|     samsung|01-01-2023|1100000|                NULL|         NULL|
|         2|     samsung|01-02-2023|1120000|             1100000|         1.79|
|         2|     samsung|01-03-2023|1080000|             1120000|         -3.7|
|         2|     samsung|01-04-2023|1800

In [29]:
window = Window.partitionBy('product_id').orderBy('sales_date')
last_month_df =df.withColumn('previous_month_sales', lead(col('sales'),1).over(window))
last_month_df.show()

+----------+------------+----------+-------+--------------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|
+----------+------------+----------+-------+--------------------+
|         1|      iphone|01-01-2023|1500000|             1300000|
|         1|      iphone|01-02-2023|1300000|             1600000|
|         1|      iphone|01-03-2023|1600000|             1700000|
|         1|      iphone|01-04-2023|1700000|             1200000|
|         1|      iphone|01-05-2023|1200000|             1100000|
|         1|      iphone|01-06-2023|1100000|                NULL|
|         2|     samsung|01-01-2023|1100000|             1120000|
|         2|     samsung|01-02-2023|1120000|             1080000|
|         2|     samsung|01-03-2023|1080000|             1800000|
|         2|     samsung|01-04-2023|1800000|              980000|
|         2|     samsung|01-05-2023| 980000|             1100000|
|         2|     samsung|01-06-2023|1100000|                NULL|
|         

In [26]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

emp_schema=['id','name','salary','dept','gender']

emp_df = spark.createDataFrame(data=emp_data,schema=emp_schema)
emp_df = emp_df.select('id','name','salary','gender','dept')
emp_df.show()

+---+--------+------+------+---------+
| id|    name|salary|gender|     dept|
+---+--------+------+------+---------+
|  1|  manish| 50000|     m|       IT|
|  2|  vikash| 60000|     m|    sales|
|  3| raushan| 70000|     m|marketing|
|  4|  mukesh| 80000|     m|       IT|
|  5|   priti| 90000|     f|    sales|
|  6|  nikita| 45000|     f|marketing|
|  7|  ragini| 55000|     f|marketing|
|  8|   rashi|100000|     f|       IT|
|  9|  aditya| 65000|     m|       IT|
| 10|   rahul| 50000|     m|marketing|
| 11|   rakhi| 50000|     f|       IT|
| 12|akhilesh| 90000|     m|    sales|
+---+--------+------+------+---------+



# Range and row between

In [52]:
product_data = [
(2,"samsung","01-01-1995",11000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-01-2006",15000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(3,"oneplus","01-01-2010",23000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]
ms = ['product_id', 'product_name', 'sales_date', 'sales']
rgdf = spark.createDataFrame(data = product_data, schema = ms)
rgdf.show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         2|     samsung|01-01-1995|  11000|
|         1|      iphone|01-02-2023|1300000|
|         2|     samsung|01-02-2023|1120000|
|         3|     oneplus|01-02-2023|1120000|
|         1|      iphone|01-03-2023|1600000|
|         2|     samsung|01-03-2023|1080000|
|         3|     oneplus|01-03-2023|1160000|
|         1|      iphone|01-01-2006|  15000|
|         1|      iphone|01-04-2023|1700000|
|         2|     samsung|01-04-2023|1800000|
|         3|     oneplus|01-04-2023|1170000|
|         1|      iphone|01-05-2023|1200000|
|         2|     samsung|01-05-2023| 980000|
|         3|     oneplus|01-05-2023|1175000|
|         1|      iphone|01-06-2023|1100000|
|         3|     oneplus|01-01-2010|  23000|
|         2|     samsung|01-06-2023|1100000|
|         3|     oneplus|01-06-2023|1200000|
+----------+------------+----------+-------+



In [53]:
window = Window.partitionBy('product_id').orderBy('sales_date')\
       .rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)

In [54]:
rgdf.withColumn('first_sales', first('sales').over(window))\
    .withColumn('latest_sales', last('sales').over(window)).show()

+----------+------------+----------+-------+-----------+------------+
|product_id|product_name|sales_date|  sales|first_sales|latest_sales|
+----------+------------+----------+-------+-----------+------------+
|         1|      iphone|01-01-2006|  15000|      15000|     1100000|
|         1|      iphone|01-02-2023|1300000|      15000|     1100000|
|         1|      iphone|01-03-2023|1600000|      15000|     1100000|
|         1|      iphone|01-04-2023|1700000|      15000|     1100000|
|         1|      iphone|01-05-2023|1200000|      15000|     1100000|
|         1|      iphone|01-06-2023|1100000|      15000|     1100000|
|         2|     samsung|01-01-1995|  11000|      11000|     1100000|
|         2|     samsung|01-02-2023|1120000|      11000|     1100000|
|         2|     samsung|01-03-2023|1080000|      11000|     1100000|
|         2|     samsung|01-04-2023|1800000|      11000|     1100000|
|         2|     samsung|01-05-2023| 980000|      11000|     1100000|
|         2|     sam

In [56]:
# different & distinct ??

In [57]:
emp_data = [(1,"manish","11-07-2023","10:20"),
        (1,"manish","11-07-2023","11:20"),
        (2,"rajesh","11-07-2023","11:20"),
        (1,"manish","11-07-2023","11:50"),
        (2,"rajesh","11-07-2023","13:20"),
        (1,"manish","11-07-2023","19:20"),
        (2,"rajesh","11-07-2023","17:20"),
        (1,"manish","12-07-2023","10:32"),
        (1,"manish","12-07-2023","12:20"),
        (3,"vikash","12-07-2023","09:12"),
        (1,"manish","12-07-2023","16:23"),
        (3,"vikash","12-07-2023","18:08")]

emp_schema = ["id", "name", "date", "time"]
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

emp_df.show()

+---+------+----------+-----+
| id|  name|      date| time|
+---+------+----------+-----+
|  1|manish|11-07-2023|10:20|
|  1|manish|11-07-2023|11:20|
|  2|rajesh|11-07-2023|11:20|
|  1|manish|11-07-2023|11:50|
|  2|rajesh|11-07-2023|13:20|
|  1|manish|11-07-2023|19:20|
|  2|rajesh|11-07-2023|17:20|
|  1|manish|12-07-2023|10:32|
|  1|manish|12-07-2023|12:20|
|  3|vikash|12-07-2023|09:12|
|  1|manish|12-07-2023|16:23|
|  3|vikash|12-07-2023|18:08|
+---+------+----------+-----+



In [59]:
emp_df = emp_df.withColumn('timestamp', 
                          from_unixtime(unix_timestamp(expr("concat(date,' ',time)"), "dd-MM-yyy HH:mm")))

In [60]:
emp_df.show()

+---+------+----------+-----+-------------------+
| id|  name|      date| time|          timestamp|
+---+------+----------+-----+-------------------+
|  1|manish|11-07-2023|10:20|2023-07-11 10:20:00|
|  1|manish|11-07-2023|11:20|2023-07-11 11:20:00|
|  2|rajesh|11-07-2023|11:20|2023-07-11 11:20:00|
|  1|manish|11-07-2023|11:50|2023-07-11 11:50:00|
|  2|rajesh|11-07-2023|13:20|2023-07-11 13:20:00|
|  1|manish|11-07-2023|19:20|2023-07-11 19:20:00|
|  2|rajesh|11-07-2023|17:20|2023-07-11 17:20:00|
|  1|manish|12-07-2023|10:32|2023-07-12 10:32:00|
|  1|manish|12-07-2023|12:20|2023-07-12 12:20:00|
|  3|vikash|12-07-2023|09:12|2023-07-12 09:12:00|
|  1|manish|12-07-2023|16:23|2023-07-12 16:23:00|
|  3|vikash|12-07-2023|18:08|2023-07-12 18:08:00|
+---+------+----------+-----+-------------------+



In [62]:
window = Window.partitionBy('id','date').orderBy('date').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)

In [63]:
new_df = emp_df.withColumn('login', first("timestamp").over(window))\
        .withColumn('logout', last('timestamp').over(window))\
        .withColumn('login',to_timestamp('login', 'yyyy-MM-dd HH:mm:ss'))\
        .withColumn('logout', to_timestamp('logout', 'yyyy-MM-dd HH:mm:ss'))\
        .withColumn('total_time', col('logout')-col('login')).show()

+---+------+----------+-----+-------------------+-------------------+-------------------+--------------------+
| id|  name|      date| time|          timestamp|              login|             logout|          total_time|
+---+------+----------+-----+-------------------+-------------------+-------------------+--------------------+
|  1|manish|11-07-2023|10:20|2023-07-11 10:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|11:20|2023-07-11 11:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|11:50|2023-07-11 11:50:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|19:20|2023-07-11 19:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|12-07-2023|10:32|2023-07-12 10:32:00|2023-07-12 10:32:00|2023-07-12 16:23:00|INTERVAL '0 05:51...|
|  1|manish|12-07-2023|12:20|2023-07-12 12:20:00|2023-07-12 10:32:00|2023-07-12 16:23:00|INTERVAL '0 05:51...|
|

In [65]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]
emp_schema = ["product_id", "product_name", "sales_date", "sales"]
prod_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

prod_df.show()

+----------+------------+----------+-----+
|product_id|product_name|sales_date|sales|
+----------+------------+----------+-----+
|         1|      manish|11-07-2023|10:20|
|         1|      manish|11-07-2023|11:20|
|         2|      rajesh|11-07-2023|11:20|
|         1|      manish|11-07-2023|11:50|
|         2|      rajesh|11-07-2023|13:20|
|         1|      manish|11-07-2023|19:20|
|         2|      rajesh|11-07-2023|17:20|
|         1|      manish|12-07-2023|10:32|
|         1|      manish|12-07-2023|12:20|
|         3|      vikash|12-07-2023|09:12|
|         1|      manish|12-07-2023|16:23|
|         3|      vikash|12-07-2023|18:08|
+----------+------------+----------+-----+



In [68]:
window = Window.partitionBy('product_id').orderBy('sales_date').rowsBetween(-2, 0)

In [69]:
prod_df.withColumn("running_row", sum('sales').over(window))

DataFrame[product_id: bigint, product_name: string, sales_date: string, sales: string, running_row: double]

# flatten nested json in spark

In [71]:
jdf = spark.read.format('json')\
           .option('inferschema', True)\
           .option('multiline', True)\
           .load('resturant_json_data.json')
jdf.show()        

+----+-------+--------------------+-------------+-------------+-------------+------+
|code|message|         restaurants|results_found|results_shown|results_start|status|
+----+-------+--------------------+-------------+-------------+-------------+------+
|NULL|   NULL|                  []|            0|            0|            1|  NULL|
|NULL|   NULL|[{{{17066603}, b9...|         6835|           20|            1|  NULL|
|NULL|   NULL|                  []|            0|            0|            1|  NULL|
|NULL|   NULL|                  []|            0|            0|            1|  NULL|
|NULL|   NULL|[{{{17093124}, b9...|         8680|           20|            1|  NULL|
|NULL|   NULL|                  []|            0|            0|            1|  NULL|
|NULL|   NULL|                  []|            0|            0|            1|  NULL|
|NULL|   NULL|[{{{17580142}, b9...|          943|           20|            1|  NULL|
|NULL|   NULL|                  []|            0|            0|  

In [72]:
jdf.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [75]:
jdf.select('*', explode('restaurants').alias('new_restaurants')).drop('restaurants').printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- new_restaurants: struct (nullable = true)
 |    |-- restaurant: struct (nullable = true)
 |    |    |-- R: struct (nullable = true)
 |    |    |    |-- res_id: long (nullable = true)
 |    |    |-- apikey: string (nullable = true)
 |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |-- cuisines: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- deeplink: string (nullable = true)
 |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- events_url: string (nullable = true)
 |    |    |-- featured_image: string (nullable = true)
 |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |-- has_tab

In [78]:
jdf.select('*', explode('restaurants').alias('new_restaurants')).drop('restaurants')\
   .select('new_restaurants.restaurant.R.res_id').show()

+--------+
|  res_id|
+--------+
|17066603|
|17059541|
|17064405|
|17057797|
|17057591|
|17064266|
|17060516|
|17060320|
|17059060|
|17059012|
|17060869|
|17061231|
|17058534|
|17057925|
|17064031|
|17061237|
|17061253|
|17061296|
|17061205|
|17057397|
+--------+
only showing top 20 rows



In [80]:
jdf.select('*', explode('restaurants').alias('new_restaurants')).drop('restaurants')\
   .select('*','new_restaurants.restaurant.R.res_id').printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- new_restaurants: struct (nullable = true)
 |    |-- restaurant: struct (nullable = true)
 |    |    |-- R: struct (nullable = true)
 |    |    |    |-- res_id: long (nullable = true)
 |    |    |-- apikey: string (nullable = true)
 |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |-- cuisines: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- deeplink: string (nullable = true)
 |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- events_url: string (nullable = true)
 |    |    |-- featured_image: string (nullable = true)
 |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |-- has_tab

In [83]:
jdf.select('*', explode('restaurants').alias('new_restaurants')).drop('restaurants')\
   .select('*','new_restaurants.restaurant.R.res_id', 
                explode('new_restaurants.restaurant.establishment_types').alias('establishment_types_new')
                ,'new_restaurants.restaurant.name').show()

+----+-------+-------------+-------------+-------------+------+---------------+------+-----------------------+----+
|code|message|results_found|results_shown|results_start|status|new_restaurants|res_id|establishment_types_new|name|
+----+-------+-------------+-------------+-------------+------+---------------+------+-----------------------+----+
+----+-------+-------------+-------------+-------------+------+---------------+------+-----------------------+----+



In [ ]:
# explode = will remove null records 
# explode_outer = will show all records including null

In [91]:
jdf.select('*', explode('restaurants').alias('new_restaurants')).drop('restaurants')\
   .select('*','new_restaurants.restaurant.R.res_id', 
                explode_outer('new_restaurants.restaurant.establishment_types').alias('establishment_types_new')
                ,'new_restaurants.restaurant.name').drop('new_restaurants','code','message','results_found','results_shown',\
                                                        'results_start','status').show(truncate=False)

+--------+-----------------------+------------------------------------+
|res_id  |establishment_types_new|name                                |
+--------+-----------------------+------------------------------------+
|17066603|NULL                   |The Coop                            |
|17059541|NULL                   |Maggiano's Little Italy             |
|17064405|NULL                   |Tako Cheena by Pom Pom              |
|17057797|NULL                   |Bosphorous Turkish Cuisine          |
|17057591|NULL                   |Bahama Breeze Island Grille         |
|17064266|NULL                   |Hawkers Asian Street Fare           |
|17060516|NULL                   |Seasons 52 Fresh Grill              |
|17060320|NULL                   |Raglan Road Irish Pub and Restaurant|
|17059060|NULL                   |Hillstone                           |
|17059012|NULL                   |Hollerbach's Willow Tree Café       |
|17060869|NULL                   |Texas de Brazil               

# SCD2 (slowly changing dimension2)

In [93]:
customer_dim_data = [
(1,'manish','arwal','india','N','2022-09-15','2022-09-25'),
(2,'vikash','patna','india','Y','2023-08-12',None),
(3,'nikita','delhi','india','Y','2023-09-10',None),
(4,'rakesh','jaipur','india','Y','2023-06-10',None),
(5,'ayush','NY','USA','Y','2023-06-10',None),
(1,'manish','gurgaon','india','Y','2022-09-25',None),
]

customer_schema= ['id','name','city','country','active','effective_start_date','effective_end_date']

customer_dim_df = spark.createDataFrame(data= customer_dim_data,schema=customer_schema)
customer_dim_df.show()

+---+------+-------+-------+------+--------------------+------------------+
| id|  name|   city|country|active|effective_start_date|effective_end_date|
+---+------+-------+-------+------+--------------------+------------------+
|  1|manish|  arwal|  india|     N|          2022-09-15|        2022-09-25|
|  2|vikash|  patna|  india|     Y|          2023-08-12|              NULL|
|  3|nikita|  delhi|  india|     Y|          2023-09-10|              NULL|
|  4|rakesh| jaipur|  india|     Y|          2023-06-10|              NULL|
|  5| ayush|     NY|    USA|     Y|          2023-06-10|              NULL|
|  1|manish|gurgaon|  india|     Y|          2022-09-25|              NULL|
+---+------+-------+-------+------+--------------------+------------------+



In [94]:
sales_data = [

(1,1,'manish','2023-01-16','gurgaon','india',380),
(77,1,'manish','2023-03-11','bangalore','india',300),
(12,3,'nikita','2023-09-20','delhi','india',127),
(54,4,'rakesh','2023-08-10','jaipur','india',321),
(65,5,'ayush','2023-09-07','mosco','russia',765),
(89,6,'rajat','2023-08-10','jaipur','india',321)
]

sales_schema = ['sales_id', 'customer_id','customer_name', 'sales_date', 'food_delivery_address','food_delivery_country', 'food_cost']

sales_df = spark.createDataFrame(data=sales_data,schema=sales_schema)
sales_df.show()

+--------+-----------+-------------+----------+---------------------+---------------------+---------+
|sales_id|customer_id|customer_name|sales_date|food_delivery_address|food_delivery_country|food_cost|
+--------+-----------+-------------+----------+---------------------+---------------------+---------+
|       1|          1|       manish|2023-01-16|              gurgaon|                india|      380|
|      77|          1|       manish|2023-03-11|            bangalore|                india|      300|
|      12|          3|       nikita|2023-09-20|                delhi|                india|      127|
|      54|          4|       rakesh|2023-08-10|               jaipur|                india|      321|
|      65|          5|        ayush|2023-09-07|                mosco|               russia|      765|
|      89|          6|        rajat|2023-08-10|               jaipur|                india|      321|
+--------+-----------+-------------+----------+---------------------+-------------

In [95]:
joined_data = customer_dim_df.join(sales_df, customer_dim_df['id'] == sales_df['customer_id'], 'left')

In [97]:
joined_data.show()

+---+------+-------+-------+------+--------------------+------------------+--------+-----------+-------------+----------+---------------------+---------------------+---------+
| id|  name|   city|country|active|effective_start_date|effective_end_date|sales_id|customer_id|customer_name|sales_date|food_delivery_address|food_delivery_country|food_cost|
+---+------+-------+-------+------+--------------------+------------------+--------+-----------+-------------+----------+---------------------+---------------------+---------+
|  1|manish|  arwal|  india|     N|          2022-09-15|        2022-09-25|      77|          1|       manish|2023-03-11|            bangalore|                india|      300|
|  1|manish|  arwal|  india|     N|          2022-09-15|        2022-09-25|       1|          1|       manish|2023-01-16|              gurgaon|                india|      380|
|  2|vikash|  patna|  india|     Y|          2023-08-12|              NULL|    NULL|       NULL|         NULL|      NULL

In [98]:
# Identify changes in address and create new records.

In [110]:
new_records_df = joined_data.where(
                   (col('food_delivery_address') != col('city')) & (col('active') == 'Y'))\
                 .withColumn('active', lit('Y'))\
                 .withColumn('effective_start_date', col('sales_date'))\
                 .withColumn('effective_end_date', lit(None))\
                 .select('customer_id','customer_name',
                        col('food_delivery_address').alias('city'),
                        'food_delivery_country','active', 'effective_start_date','effective_end_date')
new_records_df.show()      

+-----------+-------------+---------+---------------------+------+--------------------+------------------+
|customer_id|customer_name|     city|food_delivery_country|active|effective_start_date|effective_end_date|
+-----------+-------------+---------+---------------------+------+--------------------+------------------+
|          1|       manish|bangalore|                india|     Y|          2023-03-11|              NULL|
|          5|        ayush|    mosco|               russia|     Y|          2023-09-07|              NULL|
+-----------+-------------+---------+---------------------+------+--------------------+------------------+



In [111]:
#update old records

In [117]:
old_records = joined_data.where(
                   (col('food_delivery_address') != col('city')) & (col('active') == 'Y'))\
                 .withColumn('active', lit('N'))\
                 .withColumn('effective_end_date', col('sales_date'))\
                 .select('customer_id',
                         'customer_name',
                         'city',
                         'food_delivery_country',
                         'active', 
                         'effective_start_date',
                         'effective_end_date')
old_records.show()    

+-----------+-------------+-------+---------------------+------+--------------------+------------------+
|customer_id|customer_name|   city|food_delivery_country|active|effective_start_date|effective_end_date|
+-----------+-------------+-------+---------------------+------+--------------------+------------------+
|          1|       manish|gurgaon|                india|     N|          2022-09-25|        2023-03-11|
|          5|        ayush|     NY|               russia|     N|          2023-06-10|        2023-09-07|
+-----------+-------------+-------+---------------------+------+--------------------+------------------+



In [118]:
# find out new customers and insert them 

In [119]:
new_customers = sales_df.join(customer_dim_df, sales_df['customer_id'] == customer_dim_df['id'], 'leftanti')\
                 .withColumn('active', lit('Y'))\
                 .withColumn('effective_start_date', col('sales_date'))\
                 .withColumn('effective_end_date', lit(None))\
                 .select('customer_id','customer_name',
                         'food_delivery_address',
                         'food_delivery_country',
                         'active', 
                         'effective_start_date',
                         'effective_end_date')
new_customers.show()   

+-----------+-------------+---------------------+---------------------+------+--------------------+------------------+
|customer_id|customer_name|food_delivery_address|food_delivery_country|active|effective_start_date|effective_end_date|
+-----------+-------------+---------------------+---------------------+------+--------------------+------------------+
|          6|        rajat|               jaipur|                india|     Y|          2023-08-10|              NULL|
+-----------+-------------+---------------------+---------------------+------+--------------------+------------------+



In [120]:
# Merge all df in one df

In [121]:
final_records = customer_dim_df.union(new_records_df).union(old_records).union(new_customers)
final_records.show()

+---+------+---------+-------+------+--------------------+------------------+
| id|  name|     city|country|active|effective_start_date|effective_end_date|
+---+------+---------+-------+------+--------------------+------------------+
|  1|manish|    arwal|  india|     N|          2022-09-15|        2022-09-25|
|  2|vikash|    patna|  india|     Y|          2023-08-12|              NULL|
|  3|nikita|    delhi|  india|     Y|          2023-09-10|              NULL|
|  4|rakesh|   jaipur|  india|     Y|          2023-06-10|              NULL|
|  5| ayush|       NY|    USA|     Y|          2023-06-10|              NULL|
|  1|manish|  gurgaon|  india|     Y|          2022-09-25|              NULL|
|  1|manish|bangalore|  india|     Y|          2023-03-11|              NULL|
|  5| ayush|    mosco| russia|     Y|          2023-09-07|              NULL|
|  1|manish|  gurgaon|  india|     N|          2022-09-25|        2023-03-11|
|  5| ayush|       NY| russia|     N|          2023-06-10|      

In [122]:
# Remove duplicate data

In [123]:
window = Window.partitionBy('id','active').orderBy(col('effective_start_date').desc())

In [124]:
final_records.withColumn('rnk', rank().over(window))\
             .filter(~((col('active')=='Y')&(col('rnk')>=2))).drop('rnk').show()

+---+------+---------+-------+------+--------------------+------------------+
| id|  name|     city|country|active|effective_start_date|effective_end_date|
+---+------+---------+-------+------+--------------------+------------------+
|  1|manish|  gurgaon|  india|     N|          2022-09-25|        2023-03-11|
|  1|manish|    arwal|  india|     N|          2022-09-15|        2022-09-25|
|  1|manish|bangalore|  india|     Y|          2023-03-11|              NULL|
|  2|vikash|    patna|  india|     Y|          2023-08-12|              NULL|
|  3|nikita|    delhi|  india|     Y|          2023-09-10|              NULL|
|  4|rakesh|   jaipur|  india|     Y|          2023-06-10|              NULL|
|  5| ayush|       NY| russia|     N|          2023-06-10|        2023-09-07|
|  5| ayush|    mosco| russia|     Y|          2023-09-07|              NULL|
|  6| rajat|   jaipur|  india|     Y|          2023-08-10|              NULL|
+---+------+---------+-------+------+--------------------+------